In [132]:
import pandas as pd
import math
import os


In [133]:
# Define the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 3963   # Radius of the Earth in kilometers
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + \
        math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
        math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c# * 0.621371  # Convert kilometers to miles
    return distance



In [134]:
cust=pd.read_csv('DSB Customer Locations.csv')
branch=pd.read_csv('DSB Branches.csv')

In [135]:
branch

,Branch,Branch Long,Branch Lat
0,London,-0.109863,51.481383
1,Oxford,-1.252441,51.727028
2,Nottingham,-1.142578,52.948638
3,Newcastle,-1.604004,54.965002


In [136]:
cust['branch'] = ''
cust['distance']=''
cust['branch long']=''
cust['branch lat']=''

In [137]:
for i in cust.index:
    lon1 = cust.iloc[i,1]
    lat1 = cust.iloc[i,2]
    shortest=10000000
    branch_name=''
    for b in branch.index:
        lon2= branch.iloc[b,1]
        lat2= branch.iloc[b,2]
        distance=round(haversine(lat1, lon1, lat2, lon2),2)
        if distance<shortest:
            shortest = distance
            branch_name = branch.iloc[b,0]

        # print('Customer', str(cust.iloc[i,0]),',','Branch',branch.iloc[b,0],',',str(distance))
    cust.iloc[i,3]= branch_name
    cust.iloc[i,4]= shortest
    cust.iloc[i,5]= lon2
    cust.iloc[i,6]= lat2

    # print(shortest,branch_name)

In [138]:
cust = cust.sort_values(['branch','distance']).set_index(['branch','distance']).copy()

In [139]:
cust['priority'] = 1
p = cust.groupby(level=[0]).cumsum(numeric_only=True)['priority']
cust['priority']= p

In [144]:
cust.columns=[i.lower().strip().replace(' ','_') for i in cust.columns]

In [152]:
cust.reset_index()[['branch', 'customer', 'priority','distance',  'address_long', 'address_lat','branch_long', 'branch_lat']].to_csv('cust.csv',index=False)

In [153]:
os.startfile('cust.csv')